# AI - Project 01- Mohsen Fayyaz - 810196650

<ul>
    <li><b>Modeling:</b><br>
        <ul>
            <li>
                <b>Initial State:</b> map read from the file
            </li>
            <li>
                <b>Goal State:</b> a map which doesn't contain any numbers in it i.e. 1,2 or 3.
            </li>
            <li>
                <b>actions:</b> There are 8 different possible actions in any state which are moving P or Q one block to one of the four cardinal directions. (The move might not be possible due to colliding with walls or etc.)
            </li>
        </ul>
    </li>
</ul>

In [10]:
from queue import Queue
from time import sleep, time
import copy
from os import system, name
import sys
from IPython.display import clear_output

P_CHAR = "P"
Q_CHAR = "Q"
WALL_CHAR = "%"
EMPTY_CHAR = " "
P_FOOD = "1"
Q_FOOD = "2"
BOTH_FOOD = "3"


class Pac_map_handler:
    @staticmethod
    def find_in_map(the_map, char):
        for line in the_map:
            if char in line:
                return the_map.index(line), line.index(char)

    @staticmethod
    def is_in_map(pac_map: list, row, col):
        return 0 <= row < len(pac_map) and 0 <= col < len(pac_map[0])

    @staticmethod
    def can_p_goto(pac_map: list, row, col):
        return Pac_map_handler.is_in_map(pac_map, row, col) and pac_map[row][col] in {P_FOOD, EMPTY_CHAR, BOTH_FOOD}

    @staticmethod
    def can_q_goto(pac_map: list, row, col):
        return Pac_map_handler.is_in_map(pac_map, row, col) and pac_map[row][col] in {Q_FOOD, EMPTY_CHAR, BOTH_FOOD}

    # P MOVEMENT
    @staticmethod
    def move_p_right(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row, col + 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col + 1] = P_CHAR
        return pac_map

    @staticmethod
    def move_p_left(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row, col - 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col - 1] = P_CHAR
        return pac_map

    @staticmethod
    def move_p_up(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row - 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row - 1][col] = P_CHAR
        return pac_map

    @staticmethod
    def move_p_down(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row + 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row + 1][col] = P_CHAR
        return pac_map

    # Q MOVEMENT
    @staticmethod
    def move_q_right(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row, col + 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col + 1] = Q_CHAR
        return pac_map

    @staticmethod
    def move_q_left(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row, col - 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col - 1] = Q_CHAR
        return pac_map

    @staticmethod
    def move_q_up(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row - 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row - 1][col] = Q_CHAR
        return pac_map

    @staticmethod
    def move_q_down(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row + 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row + 1][col] = Q_CHAR
        return pac_map

    @staticmethod
    def print_map(pac_map):
        cls()
        print('\n'.join(''.join(item) for item in pac_map))


def cls():
    # for windows
    if name == 'nt':
        _ = system('cls')

        # for mac and linux(here, os.name is 'posix')
    else:
        _ = system('clear')

    clear_output(wait=True)

In [11]:
class State:
    def __init__(self, map_data, parent=None):
        self.map_data = map_data
        self.parent = parent
        self.hash = ''.join(''.join(item) for item in map_data).replace("%", "")

    def get_hash(self):
        return self.hash

    def get_map(self):
        return self.map_data

    def get_parent(self):
        return self.parent


# BFS)
<ul>
    <li>
        <b>Algorithm:</b> Breadth-first search is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root, and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.
        <br>
        For any new state we make, we add another node to frontier and also the node's hash into explored set. That's because in BFS we can be sure that if we see a new state which was added into the frontier or explored set before, it doesn't have to be added to the frontier again.(This assumption is not true for A* algorithm)
    </li>

</ul>


## BFS implementation:

In [12]:
class BFS:
    def __init__(self, pac_map):
        self.start_state = State(pac_map, None)
        self.explored_states_hash = set()
        self.unique_states_hash = set()
        self.num_of_explored_states = 0
        self.num_of_unique_explored_states = 0
        self.frontier_states = Queue()
        self.goal_state = None
        self.goal_state_is_found = False

    def start(self):
        self.goal_state_is_found = False
        self.frontier_states = Queue()
        self.frontier_states.put(self.start_state)
        self.explored_states_hash = set()
        self.explored_states_hash.add(self.start_state.get_hash())
        self.unique_states_hash = set()
        self.num_of_explored_states = 0
        self.num_of_unique_explored_states = 0
        while not self.frontier_states.empty():
            current_state = self.frontier_states.get()
            self.num_of_explored_states += 1
            if current_state.get_hash() not in self.unique_states_hash:
                self.unique_states_hash.add(current_state.get_hash())
                self.num_of_unique_explored_states += 1

            # self.print_map(current_state.get_map())
            if self.are_constraints_satisfied(current_state):
                self.goal_state = current_state
                self.goal_state_is_found = True
                return True

            self.do_actions(current_state)
            if self.goal_state_is_found:
                return True

        return False

    def print_solution(self, delay=0.2):
        if self.goal_state is None:
            print("No Solution!")
        else:
            current_state = self.goal_state
            goal_depth = 0
            states_list = list()
            while current_state is not None:
                goal_depth += 1
                states_list.append(current_state)
                current_state = current_state.get_parent()

            for state in reversed(states_list):
                Pac_map_handler.print_map(state.get_map())
                sleep(delay)

            print("Explored States: " + str(self.num_of_explored_states))
            print("Explored Unique States: " + str(self.num_of_unique_explored_states))
            print("Goal Depth: " + str(goal_depth))

    def are_constraints_satisfied(self, state: State):
        for line in state.get_map():
            if P_FOOD in line or Q_FOOD in line or BOTH_FOOD in line:
                return False
        return True

    def do_actions(self, current_state):
        pac_map = current_state.get_map()
        p_row, p_col = Pac_map_handler.find_in_map(pac_map, P_CHAR)
        q_row, q_col = Pac_map_handler.find_in_map(pac_map, Q_CHAR)
        # P MOVEMENT
        self.move_p(current_state, pac_map, p_row, p_col)
        # Q MOVEMENT
        self.move_q(current_state, pac_map, q_row, q_col)

    def move_p(self, current_state, pac_map, row, col):
        new_map = Pac_map_handler.move_p_up(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_p_left(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_p_down(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_p_right(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

    def move_q(self, current_state, pac_map, row, col):
        new_map = Pac_map_handler.move_q_up(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_q_left(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_q_down(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_q_right(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

    def add_to_frontier(self, new_map, current_state):
        new_state = State(new_map, current_state)
        if not new_state.get_hash() in self.explored_states_hash:
            self.frontier_states.put(new_state)
            self.explored_states_hash.add(new_state.get_hash())
            if self.are_constraints_satisfied(new_state):
                self.goal_state = new_state
                self.goal_state_is_found = True


In [14]:
def main():
    with open('test5', 'r') as file:
        pac_map = file.read().splitlines()

    characterized_map = list()
    for line in pac_map:
        characterized_map.append(list(line))

    print("Loading...")
    start = time()

    my_bfs = BFS(characterized_map)
    my_bfs.start()
    my_bfs.print_solution(delay=0.1)

    end = time()
    print("Time: " + str(end - start) + "s")


main()


%%%%%%
%    %
%    %
%    %
%% %%%
%   Q%
%P   %
%%% %%
%    %
%%%%%% 
Explored States: 255
Explored Unique States: 255
Goal Depth: 14
Time: 2.108876943588257s


# IDS)
Iterative Deepening Search

<ul>
    <li>
        <b>Algorithm:</b> In computer science, iterative deepening search or more specifically iterative deepening depth-first search is a state space/graph search strategy in which a depth-limited version of depth-first search is run repeatedly with increasing depth limits until the goal is found.
        <br>
        For optimizing the time and number of states explored in this method, I save a number beside the explored state, which is the best depth that one branch of DFS could get into that state with.
        <br>
        This way, branches of DFS won't expand in states that have been expanded before with shorter or equal path lengths to that point.
    </li>

</ul>



## IDS implementation:

In [58]:
from State import State
from queue import Queue
from time import sleep
import copy
from Pac_map_handler import Pac_map_handler

P_CHAR = "P"
Q_CHAR = "Q"
WALL_CHAR = "%"
EMPTY_CHAR = " "
P_FOOD = "1"
Q_FOOD = "2"
BOTH_FOOD = "3"


class IDS:
    def __init__(self, pac_map):
        self.start_state = State(pac_map, None)
        self.explored_states_hash = set()
        self.unique_states_hash = set()
        self.num_of_explored_states = 0
        self.num_of_unique_explored_states = 0
        self.goal_state = None
        self.goal_state_is_found = False

    def init_attributes(self):
        self.goal_state_is_found = False
        self.explored_states_hash = set()
        # self.unique_states_hash = set()
        # self.num_of_explored_states = 0
        # self.num_of_unique_explored_states = 0

    def count_explored_state(self, state):
        self.num_of_explored_states += 1;
        if state.get_hash() not in self.unique_states_hash:
            self.unique_states_hash.add(state.get_hash())
            self.num_of_unique_explored_states += 1

    def DFS(self, state, max_depth, current_depth=1):
        self.count_explored_state(state)

        if max_depth == 14:
            Pac_map_handler.print_map(state.get_map())
            print(current_depth)
#             sleep(0.05)
            
        if current_depth > max_depth:
            return False

        if self.are_constraints_satisfied(state):
            self.goal_state_is_found = True
            self.goal_state = state
            return True

        if state.get_hash() in self.explored_states_hash:
            return False
        else:
            self.explored_states_hash.add(state.get_hash())

        pac_map = state.get_map()
        p_row, p_col = Pac_map_handler.find_in_map(pac_map, P_CHAR)
        q_row, q_col = Pac_map_handler.find_in_map(pac_map, Q_CHAR)

        # P -------
        new_map = Pac_map_handler.move_p_up(copy.deepcopy(pac_map), p_row, p_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        new_map = Pac_map_handler.move_p_left(copy.deepcopy(pac_map), p_row, p_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        new_map = Pac_map_handler.move_p_down(copy.deepcopy(pac_map), p_row, p_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        new_map = Pac_map_handler.move_p_right(copy.deepcopy(pac_map), p_row, p_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        # Q -------------
        new_map = Pac_map_handler.move_q_up(copy.deepcopy(pac_map), q_row, q_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        new_map = Pac_map_handler.move_q_left(copy.deepcopy(pac_map), q_row, q_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        new_map = Pac_map_handler.move_q_down(copy.deepcopy(pac_map), q_row, q_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        new_map = Pac_map_handler.move_q_right(copy.deepcopy(pac_map), q_row, q_col)
        new_state = State(new_map, state)
        if self.DFS(new_state, max_depth, current_depth + 1):
            return True

        return False

    def start(self):
        ids_max_depth = 0
        self.unique_states_hash = set()
        self.num_of_explored_states = 0
        self.num_of_unique_explored_states = 0
        print("Current IDS Depth:", end=" ");
        while True:
            ids_max_depth += 1
            print(ids_max_depth, end=" ->\n ")
            self.init_attributes()

            self.DFS(self.start_state, ids_max_depth)
            if self.goal_state_is_found:
                return True

        return False

    def print_solution(self, delay=0.2):
        if self.goal_state is None:
            print("No Solution!")
        else:
            current_state = self.goal_state
            states_list = list()
            while current_state is not None:
                states_list.append(current_state)
                current_state = current_state.get_parent()

            for state in reversed(states_list):
                Pac_map_handler.print_map(state.get_map())
                sleep(delay)

            print("Explored States: " + str(self.num_of_explored_states))
            print("Explored Unique States: " + str(self.num_of_unique_explored_states))
            print("Goal Depth: " + str(len(states_list)))

    def are_constraints_satisfied(self, state: State):
        for line in state.get_map():
            if P_FOOD in line or Q_FOOD in line or BOTH_FOOD in line:
                return False
        return True


In [62]:
def main():
    with open('test5', 'r') as file:
        pac_map = file.read().splitlines()

    characterized_map = list()
    for line in pac_map:
        characterized_map.append(list(line))

    print("Loading...")
    start = time()

    # my_bfs = BFS(characterized_map)
    # my_bfs.start()
    # my_bfs.print_solution(delay=0.2)

    my_ids = IDS(characterized_map)
    my_ids.start()
    my_ids.print_solution(delay=0.2)

    end = time()
    print("Time: " + str(end - start) + "s")


main()

%%%%%%
%    %
%    %
%    %
%% %%%
%  QP%
%    %
%%% %%
%    %
%%%%%% 
Explored States: 14056
Explored Unique States: 433
Goal Depth: 25
Time: 34.19281244277954s


# A* search algorithm)

<ul>
    <li>
        <b>Algorithm:</b> In computer science, A* is a computer algorithm that is widely used in pathfinding and graph traversal, which is the process of finding a path between multiple points, called "nodes". It enjoys widespread use due to its performance and accuracy.
    </li>
    <br>
    <li>
        <b>Heuristic:</b>
    </li>
</ul>



# Results

<table style="width:100%;">
    <tr></tr>
    <tr>
        <td>
            <table style="width:100%; border: 1px solid #ddd;">
                <thead>
                    <tr>
                        <th style="text-align: center;">TEST #1</th>
                    </tr>
                <tr>
                    <th style="text-align: center;">Algorithm</th>
                    <th style="text-align: center;">Goal Depth</th>
                    <th style="text-align: center;">Explored States</th>
                    <th style="text-align: center;">Unique Explored States</th>
                    <th style="text-align: center;">Execution Time</th>
                </tr>
                </thead>
                <tr>
                    <td style="text-align: center;"><b>BFS</b></td>         
                    <td style="text-align: center;">14</td>
                    <td style="text-align: center;">133595</td>
                    <td style="text-align: center;">133595</td>
                    <td style="text-align: center;">2.04s</td>
                </tr>
                <tr>
                    <td style="text-align: center;"><b>IDS</b></td>        
                    <td style="text-align: center;">The Shawshank Redemption</td>
                    <td style="text-align: center;">1994</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>        
                <tr>
                    <td style="text-align: center;"><b>A*</b></td>         
                    <td style="text-align: center;">The Godfather</td>
                    <td style="text-align: center;">1972</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
            </table>
        </td>
        <td>
            <table style="width:100%; border: 1px solid #ddd;">
                <thead>
                    <tr>
                        <th style="text-align: center;">TEST #2</th>
                    </tr>
                <tr>
                    <th style="text-align: center;">Algorithm</th>
                    <th style="text-align: center;">Goal Depth</th>
                    <th style="text-align: center;">Explored States</th>
                    <th style="text-align: center;">Unique Explored States</th>
                    <th style="text-align: center;">Execution Time</th>
                </tr>
                </thead>
                <tr>
                    <td style="text-align: center;"><b>BFS</b></td>         
                    <td style="text-align: center;">The Godfather: Part II</td>
                    <td style="text-align: center;">1974</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
                <tr>
                    <td style="text-align: center;"><b>IDS</b></td>        
                    <td style="text-align: center;">The Shawshank Redemption</td>
                    <td style="text-align: center;">1994</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>        
                <tr>
                    <td style="text-align: center;"><b>A*</b></td>         
                    <td style="text-align: center;">The Godfather</td>
                    <td style="text-align: center;">1972</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
            </table>
        </td>
        <td>
            <table style="width:100%; border: 1px solid #ddd;">
                <thead>
                    <tr>
                        <th style="text-align: center;">TEST #3</th>
                    </tr>
                <tr>
                    <th style="text-align: center;">Algorithm</th>
                    <th style="text-align: center;">Goal Depth</th>
                    <th style="text-align: center;">Explored States</th>
                    <th style="text-align: center;">Unique Explored States</th>
                    <th style="text-align: center;">Execution Time</th>
                </tr>
                </thead>
                <tr>
                    <td style="text-align: center;"><b>BFS</b></td>         
                    <td style="text-align: center;">21</td>
                    <td style="text-align: center;">2372</td>
                    <td style="text-align: center;">2372</td>
                    <td style="text-align: center;">6.04s</td>
                </tr>
                <tr>
                    <td style="text-align: center;"><b>IDS</b></td>        
                    <td style="text-align: center;">The Shawshank Redemption</td>
                    <td style="text-align: center;">1994</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>        
                <tr>
                    <td style="text-align: center;"><b>A*</b></td>         
                    <td style="text-align: center;">The Godfather</td>
                    <td style="text-align: center;">1972</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
            </table>
        </td>
    </tr>
    <tr></tr>
    <tr>
        <td>
            <table style="width:100%; border: 1px solid #ddd;">
                <thead>
                    <tr>
                        <th style="text-align: center;">TEST #4</th>
                    </tr>
                <tr>
                    <th style="text-align: center;">Algorithm</th>
                    <th style="text-align: center;">Goal Depth</th>
                    <th style="text-align: center;">Explored States</th>
                    <th style="text-align: center;">Unique Explored States</th>
                    <th style="text-align: center;">Execution Time</th>
                </tr>
                </thead>
                <tr>
                    <td style="text-align: center;"><b>BFS</b></td>         
                    <td style="text-align: center;">The Godfather: Part II</td>
                    <td style="text-align: center;">1974</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
                <tr>
                    <td style="text-align: center;"><b>IDS</b></td>        
                    <td style="text-align: center;">The Shawshank Redemption</td>
                    <td style="text-align: center;">1994</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>        
                <tr>
                    <td style="text-align: center;"><b>A*</b></td>         
                    <td style="text-align: center;">The Godfather</td>
                    <td style="text-align: center;">1972</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
            </table>
        </td>
        <td>
            <table style="width:100%; border: 1px solid #ddd;">
                <thead>
                    <tr>
                        <th style="text-align: center;">TEST #5</th>
                    </tr>
                <tr>
                    <th style="text-align: center;">Algorithm</th>
                    <th style="text-align: center;">Goal Depth</th>
                    <th style="text-align: center;">Explored States</th>
                    <th style="text-align: center;">Unique Explored States</th>
                    <th style="text-align: center;">Execution Time</th>
                </tr>
                </thead>
                <tr>
                    <td style="text-align: center;"><b>BFS</b></td>         
                    <td style="text-align: center;">The Godfather: Part II</td>
                    <td style="text-align: center;">1974</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
                <tr>
                    <td style="text-align: center;"><b>IDS</b></td>        
                    <td style="text-align: center;">The Shawshank Redemption</td>
                    <td style="text-align: center;">1994</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>        
                <tr>
                    <td style="text-align: center;"><b>A*</b></td>         
                    <td style="text-align: center;">The Godfather</td>
                    <td style="text-align: center;">1972</td>
                    <td style="text-align: center;">123</td>
                    <td style="text-align: center;">123</td>
                </tr>
            </table>
        </td>
    </tr>
</table>